
### Requisitos para o arquivo torneios.csv

Objetivo: realizar processo de ETL e disponibilizar os dados em camada adequada.

1. Ler o arquivo csv do Datalake [ok]
2. Renomear colunas para português usando snake case [ok]
3. Definir a tipagem de colunas correta [ok]
4. Coluna com data de cosumo do arquivo [ok]
5. Salvar dados em formato parquet na camada adequada [ok]

In [2]:
%run ../../Configuração-e-Utilitários/Variaveis.ipynb

In [3]:
%run ../../Configuração-e-Utilitários/Funções.ipynb


### Lendo os dados do arquivo

In [6]:
torneios_df = (spark.read
               .option("encoding", "UTF-16")
               .option("header", "true") # Detecção do cabeçalho
               .option("inferSchema", "true") # Detecção do tipo de dados
               .csv(f"{caminho_bronze}/torneios.csv") # Caminho do arquivo
)
torneios_df.show()

+---------+-----------+------------+------------+-----------+------------+----------+--------------+
|league_id|league_name|country_name|country_code|season_year|season_start|season_end|season_current|
+---------+-----------+------------+------------+-----------+------------+----------+--------------+
|       71|    Serie A|      Brazil|          BR|       2010|  2010-05-08|2010-11-28|        False�|
|       71|    Serie A|      Brazil|          BR|       2011|  2011-05-21|2011-12-04|        False�|
|       71|    Serie A|      Brazil|          BR|       2012|  2012-05-19|2012-12-02|        False�|
|       71|    Serie A|      Brazil|          BR|       2013|  2013-05-25|2013-12-08|        False�|
|       71|    Serie A|      Brazil|          BR|       2014|  2014-04-19|2014-12-07|        False�|
|       71|    Serie A|      Brazil|          BR|       2015|  2015-05-09|2015-12-06|        False�|
|       71|    Serie A|      Brazil|          BR|       2016|  2016-05-14|2016-12-11|      

In [7]:
torneios_df.printSchema()

root
 |-- league_id: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- season_year: integer (nullable = true)
 |-- season_start: date (nullable = true)
 |-- season_end: date (nullable = true)
 |-- season_current: string (nullable = true)



### Renomeando colunas para português

In [8]:
torneios_df = torneios_df \
    .withColumnRenamed("league_id", "id_torneio") \
    .withColumnRenamed("league_name", "nome_torneio") \
    .withColumnRenamed("country_name", "pais") \
    .withColumnRenamed("country_code", "sigla_do_pais") \
    .withColumnRenamed("season_year", "ano_da_temporada") \
    .withColumnRenamed("season_start", "dt_inicio_da_temporada") \
    .withColumnRenamed("season_end", "dt_fim_da_temporada") \
    .withColumnRenamed("season_current", "temporada_atual")

torneios_df.show()

+----------+------------+------+-------------+----------------+----------------------+-------------------+---------------+
|id_torneio|nome_torneio|  pais|sigla_do_pais|ano_da_temporada|dt_inicio_da_temporada|dt_fim_da_temporada|temporada_atual|
+----------+------------+------+-------------+----------------+----------------------+-------------------+---------------+
|        71|     Serie A|Brazil|           BR|            2010|            2010-05-08|         2010-11-28|         False�|
|        71|     Serie A|Brazil|           BR|            2011|            2011-05-21|         2011-12-04|         False�|
|        71|     Serie A|Brazil|           BR|            2012|            2012-05-19|         2012-12-02|         False�|
|        71|     Serie A|Brazil|           BR|            2013|            2013-05-25|         2013-12-08|         False�|
|        71|     Serie A|Brazil|           BR|            2014|            2014-04-19|         2014-12-07|         False�|
|        71|    

### Alterando valores booleanos para 'Sim'/'Não'

In [9]:
from pyspark.sql.functions import when, substring

torneios_df = torneios_df.withColumn('temporada_atual', when(substring(torneios_df.temporada_atual, 0, 1) == 'T', 'Sim').otherwise('Não'))

torneios_df.show()

+----------+------------+------+-------------+----------------+----------------------+-------------------+---------------+
|id_torneio|nome_torneio|  pais|sigla_do_pais|ano_da_temporada|dt_inicio_da_temporada|dt_fim_da_temporada|temporada_atual|
+----------+------------+------+-------------+----------------+----------------------+-------------------+---------------+
|        71|     Serie A|Brazil|           BR|            2010|            2010-05-08|         2010-11-28|            Não|
|        71|     Serie A|Brazil|           BR|            2011|            2011-05-21|         2011-12-04|            Não|
|        71|     Serie A|Brazil|           BR|            2012|            2012-05-19|         2012-12-02|            Não|
|        71|     Serie A|Brazil|           BR|            2013|            2013-05-25|         2013-12-08|            Não|
|        71|     Serie A|Brazil|           BR|            2014|            2014-04-19|         2014-12-07|            Não|
|        71|    

### Inserindo data de ingestão

In [10]:
# Função para adição de data de ingestão
torneios_df = adicionar_data_de_ingestao(
    df = torneios_df,
    nome_coluna = 'data_de_ingestao',
    incluir_hora = False,
    add_tempo = -3,
    unidade_de_medida = 'horas'
)

torneios_df.show()

+----------+------------+------+-------------+----------------+----------------------+-------------------+---------------+----------------+
|id_torneio|nome_torneio|  pais|sigla_do_pais|ano_da_temporada|dt_inicio_da_temporada|dt_fim_da_temporada|temporada_atual|data_de_ingestao|
+----------+------------+------+-------------+----------------+----------------------+-------------------+---------------+----------------+
|        71|     Serie A|Brazil|           BR|            2010|            2010-05-08|         2010-11-28|            Não|      2024-05-01|
|        71|     Serie A|Brazil|           BR|            2011|            2011-05-21|         2011-12-04|            Não|      2024-05-01|
|        71|     Serie A|Brazil|           BR|            2012|            2012-05-19|         2012-12-02|            Não|      2024-05-01|
|        71|     Serie A|Brazil|           BR|            2013|            2013-05-25|         2013-12-08|            Não|      2024-05-01|
|        71|     Ser

In [14]:
torneios_df.printSchema()

root
 |-- id_torneio: string (nullable = true)
 |-- nome_torneio: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- sigla_do_pais: string (nullable = true)
 |-- ano_da_temporada: integer (nullable = true)
 |-- dt_inicio_da_temporada: date (nullable = true)
 |-- dt_fim_da_temporada: date (nullable = true)
 |-- temporada_atual: string (nullable = false)
 |-- data_de_ingestao: date (nullable = true)



### Inserindo o arquivo tratado na camada silver

In [ ]:
torneios_df.write.mode('overwrite').parquet(f"{caminho_silver}/torneios.csv")